# Imports

In [1]:
import pandas as pd
import numpy as np
import mne
import os
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import gc
import json
from tsfel import *

In [2]:
train_path = "Data/training"

#test_path = "Data/test_phase1"

test_path = "Data/test_phase2"

In [3]:
#Saving additional information that might be required while making predictions 
feature_extraction_info = {}

In [4]:
train_files = os.listdir(train_path)
train_subjects = [int(file[4:8]) for file in train_files if "EC_raw" in file]

feature_extraction_info['training_dataset_subjects'] = train_subjects

print("Training data subjects\n")
print(pd.Series(train_subjects))

Training data subjects

0          1
1          2
2          3
3          4
4          5
        ... 
1195    1196
1196    1197
1197    1198
1198    1199
1199    1200
Length: 1200, dtype: int64


In [5]:
test_files = os.listdir(test_path)
test_subjects = [int(file[4:8]) for file in test_files if "EC_raw" in file]

n_subjects_test = len(test_subjects)

feature_extraction_info['test_dataset_subjects'] = test_subjects
feature_extraction_info['n_subjects_test'] = n_subjects_test

print("Test data subjects\n")
print(pd.Series(test_subjects))

Test data subjects

0      1601
1      1602
2      1603
3      1604
4      1605
       ... 
395    1996
396    1997
397    1998
398    1999
399    2000
Length: 400, dtype: int64


# Processing Params

### Cropping data 

In [6]:
#Eyes closed - roughly 40 secs data available for all subjects
EC_crop_start = 1 #secs
EC_crop_end = 39 #secs

#Eyes open - roughly 20 secs data available for all subjects
EO_crop_start = 1 #secs
EO_crop_end = 19 #secs

### Resampling and filtering params

In [7]:
#All data will be resampled at this rate
new_sRate = 250 #Hz

#Notch filter
notch_freqz = [60.0, 120.0] #Hz

#Bandpass filter 
band_low = 1 #Hz
band_high = 100 #Hz

### Augmentation params

In [8]:
#Window size in secs
sample_duration = 6 

#window jump size secs
jump_duration = 4 

sample_size = int(new_sRate*sample_duration)
jump_size = int(new_sRate*jump_duration)

### Data split params

In [9]:
#For train-test-split
test_size = 0.15

# Loading Data

In [10]:
def load_n_process(path, condition, subject):
    """
    Loads data into a mne-raw object, from a file pertaining to a subject under a condition.
    
    Then following operations are performed on the raw object:
        1) Resampling
        2) Notch filtering
        3) IIR bandpass filtering
        
    Then data is cropped between two points in time.
    
    Returns the data back as a 2d array
    
    """
    
    fname = f"subj{subject:04}_" + condition +"_raw.fif.gz"
    raw = mne.io.read_raw(os.path.join(path,fname), preload=True, verbose=0)
    raw.resample(new_sRate)
    raw = raw.notch_filter(notch_freqz, verbose='WARNING')
    raw = raw.filter(l_freq=band_low, h_freq=band_high, method="iir", n_jobs=-1, verbose='WARNING')
    if condition == "EO":
        raw = raw.crop(tmin=EO_crop_start, tmax=EO_crop_end)
    elif condition == "EC":
        raw = raw.crop(tmin=EC_crop_start, tmax=EC_crop_end)
    else:
        print("Not a valid condition")
    raw = raw.get_data()
    return raw

In [11]:
def load_dataset(path, condition, subjects):
    
    """
    Loads and processes data from files pertaining to subjects under a condition.
    
    All files must exist in the path provided as argument
    
    Returns the data in the form of a 3d array
    
    """

    dataset = Parallel(n_jobs=-1)(delayed(load_n_process)(path, condition, s) for s in tqdm(subjects))

    dataset = np.array(dataset)

    dataset = dataset[:, :-1, :]

    dataset = 100000*dataset

    return dataset

## Loading data of Eyes closed condition (EC)

In [12]:
train_ec = load_dataset(train_path, "EC", train_subjects)

print(f"Available Training data shape for 'eyes closed' condition: {train_ec.shape}")

  0%|          | 0/1200 [00:00<?, ?it/s]

Available Training data shape for 'eyes closed' condition: (1200, 128, 9501)


In [13]:
test_ec = load_dataset(test_path, "EC", test_subjects)

print(f"Avaialable Test data shape for 'eyes closed' condition: {test_ec.shape}")

  0%|          | 0/400 [00:00<?, ?it/s]

Avaialable Test data shape for 'eyes closed' condition: (400, 128, 9501)


## Loading data of Eyes open condition (EO)

In [14]:
train_eo = load_dataset(train_path, "EO", train_subjects)

print(f"Available Training data shape for 'eyes open' condition: {train_eo.shape}")

  0%|          | 0/1200 [00:00<?, ?it/s]

Available Training data shape for 'eyes open' condition: (1200, 128, 4501)


In [15]:
test_eo = load_dataset(test_path, "EO", test_subjects)

print(f"Available Test data shape for 'eyes open' condition: {test_eo.shape}")

  0%|          | 0/400 [00:00<?, ?it/s]

Available Test data shape for 'eyes open' condition: (400, 128, 4501)


## Loading target values of training set

In [16]:
df_train_ages = pd.read_csv(os.path.join(train_path, "train_subjects.csv"), index_col=None)
train_ages = df_train_ages['age'].to_numpy()

print(df_train_ages[["id", "age"]])

        id        age
0        1   8.581679
1        2  17.324321
2        3  11.059890
3        4   6.027720
4        5  11.306297
...    ...        ...
1195  1196   7.979237
1196  1197   9.107232
1197  1198  14.886835
1198  1199   9.102213
1199  1200   6.003080

[1200 rows x 2 columns]


# Train-Test split

### Split of EC condition data

In [17]:
X_train_ec, X_valid_ec, Y_train_ec, Y_valid_ec = train_test_split(train_ec, train_ages, test_size=test_size, random_state=999)

print("Training data shape for EC condition after train-test split: ", X_train_ec.shape)
print("Validation data shape for EC condition  after train_test split: ", X_valid_ec.shape)

del(train_ec)
gc.collect()

Training data shape for EC condition after train-test split:  (1020, 128, 9501)
Validation data shape for EC condition  after train_test split:  (180, 128, 9501)


2499

### Split of EO condition data

In [18]:
X_train_eo, X_valid_eo, Y_train_eo, Y_valid_eo = train_test_split(train_eo, train_ages, test_size=test_size, random_state=999)

print("Training data shape for EO condition after train-test split: ", X_train_eo.shape)
print("Validation data shape for EO condition after train-test split: ", X_valid_eo.shape)

del(train_eo)
gc.collect()

Training data shape for EO condition after train-test split:  (1020, 128, 4501)
Validation data shape for EO condition after train-test split:  (180, 128, 4501)


21

In [19]:
#Number of subjects used for training (equal for X_train_ec and X_train_eo)
n_subjects_train = X_train_ec.shape[0]

#Number of subjects used for validation (equal for X_train_ec and X_train_eo)
n_subjects_valid = X_valid_ec.shape[0] 

#Number of channels (same for all training and validation arrays)
n_channels = X_train_ec.shape[1] 

feature_extraction_info["n_subjects_train"] = n_subjects_train
feature_extraction_info["n_subjects_valid"] = n_subjects_valid
feature_extraction_info["n_channels"] = n_channels

# Data Augmentation with sliding window 

A sliding window is used to create more samples for training and validation.
    
A window of size of length 'sample_size' is slid, with a jump length of 'jump_size', over 
the entire available epoch of a subject's data, so as to extract more samples from it.

The jump size decides the amount of overlap between any two consecutive extracted samples.

Augmentation of training dataset increases the number of training samples data, which helps models fit more robustly.

Augmentation of test dataset increases the number of sub-predictions that can be made over a single subjects data, and averaging over those sub-predictions increases the robustness of that subject's prediction.

In [20]:
def augment_training_samples(X, Y):
    
    """
   Inputs:
   1) X - Data array of shape (n_subjects, n_channels, cropped_epoch_length)
   2) Y - Data array of shape (n_subjects,)
   
   Returns:
   1) X_aug - Data array of shape (n_samples, n_channels, sample_size)
    
    """
    
    X_aug = []
    Y_aug = []

    m = X.shape[-1]

    for s in range(X.shape[0]):
        k = 0
        while (k <= m-1-sample_size):
            X_aug.append(X[s,:,k:k+sample_size])
            Y_aug.append(Y[s])
            k += jump_size

    X_aug = np.array(X_aug)
    Y_aug = np.array(Y_aug)

    return X_aug, Y_aug

In [21]:
def augment_test_samples(X):
    
    """
   Inputs:
   1) X - Data array of shape (n_subjects, n_channels, cropped_epoch_length)
   
   Returns:
   1) X_aug - Data array of shape (n_samples, n_channels, sample_size)
    
    """
    
    X_aug = []

    m = X.shape[-1]

    for s in range(X.shape[0]):
        k = 0
        while (k <= m-1-sample_size):
            X_aug.append(X[s,:,k:k+sample_size])
            k += jump_size

    X_aug = np.array(X_aug)

    return X_aug

### Augmentation of EC condition data

In [22]:
X_train_ec_aug, Y_train_ec_aug = augment_training_samples(X_train_ec, Y_train_ec)

print("Augmented training data shape for EC condition: ", X_train_ec_aug.shape)
print("Augmented training target shape for EC condition: ", Y_train_ec_aug.shape)

del(X_train_ec)
del(Y_train_ec)
gc.collect()

Augmented training data shape for EC condition:  (9180, 128, 1500)
Augmented training target shape for EC condition:  (9180,)


84

In [23]:
X_valid_ec_aug, Y_valid_ec_aug = augment_training_samples(X_valid_ec, Y_valid_ec)

print("Augmented validation data shape for EC condition: ", X_valid_ec_aug.shape)
print("Augmented validation target shape for EC condition: ", Y_valid_ec_aug.shape)

del(X_valid_ec)
del(Y_valid_ec)
gc.collect()

Augmented validation data shape for EC condition:  (1620, 128, 1500)
Augmented validation target shape for EC condition:  (1620,)


21

In [24]:
X_test_ec_aug = augment_test_samples(test_ec)

print("Augmented X_test data shape for EC condition: ", X_test_ec_aug.shape)

del(test_ec)
gc.collect()

Augmented X_test data shape for EC condition:  (3600, 128, 1500)


21

In [25]:
n_windows_ec = int(X_test_ec_aug.shape[0]/n_subjects_test)
feature_extraction_info['n_windows_ec'] = n_windows_ec

print("Number of data samples extracted from each Subject's EC data: ", n_windows_ec)

Number of data samples extracted from each Subject's EC data:  9


### Augmentation of EO condition data

In [26]:
X_train_eo_aug, Y_train_eo_aug = augment_training_samples(X_train_eo, Y_train_eo)

print("Augmented training data shape for EO condition: ", X_train_eo_aug.shape)
print("Augmented training target shape for EO condition: ", Y_train_eo_aug.shape)

del(X_train_eo)
del(Y_train_eo)
gc.collect()

Augmented training data shape for EO condition:  (4080, 128, 1500)
Augmented training target shape for EO condition:  (4080,)


42

In [27]:
X_valid_eo_aug, Y_valid_eo_aug = augment_training_samples(X_valid_eo, Y_valid_eo)

print("Augmented validation data shape for EO condition: ", X_valid_eo_aug.shape)
print("Augmented validation target shape for EO condition: ", Y_valid_eo_aug.shape)

del(X_valid_eo)
del(Y_valid_eo)
gc.collect()

Augmented validation data shape for EO condition:  (720, 128, 1500)
Augmented validation target shape for EO condition:  (720,)


21

In [28]:
X_test_eo_aug = augment_test_samples(test_eo)

print("Augmented test data shape for EO condition: ", X_test_eo_aug.shape)

del(test_eo)
gc.collect()

Augmented test data shape for EO condition:  (1600, 128, 1500)


21

In [29]:
n_windows_eo = int(X_test_eo_aug.shape[0]/n_subjects_test)
feature_extraction_info['n_windows_eo'] = n_windows_eo

print("Number of data samples extracted from each Subject's EO data: ", n_windows_eo)

Number of data samples extracted from each Subject's EO data:  4


# Feature Extraction

In [30]:
def calc_Features(signal):
    
    """
    Takes a signal (1d array), and returns a 1d array containing temporal, spectral, & statistical features 
    calculated over that signal
    """
    
    signal_abs = np.abs(signal)
    
    features = []
    
    features.append(auc(signal, new_sRate))  #Computes the area under the curve of the signal computed with trapezoid rule.
    features.append(autocorr(signal))  #Computes autocorrelation of the signal.
    features.append(calc_centroid(signal, new_sRate)) #Computes the centroid along the time axis.
    features.append(calc_mean(signal_abs))  #Computes mean value of the absolute values of the signal.
    features.append(calc_median(signal_abs))  #Computes median of the absolute values of the signal.
    features.append(calc_std(signal_abs))  #Computes standard deviation (std) of the absolute values of the signal.
    features.append(calc_var(signal_abs))  #Computes variance of the absolute values of the signal.
    features.append(distance(signal))  #Computes signal traveled distance.
    features.append(fundamental_frequency(signal, new_sRate))    #Computes fundamental frequency of the signal.
    features.append(interq_range(signal))  #Computes interquartile range of the signal.
    features.append(kurtosis(signal))  #Computes kurtosis of the signal.
    features.append(max_power_spectrum(signal, new_sRate))   #Computes maximum power spectrum density of the signal.
    features.append(mean_abs_deviation(signal))    #Computes mean absolute deviation of the signal.
    features.append(mean_abs_diff(signal))     #Computes mean absolute differences of the signal.
    features.append(mean_diff(signal))     #Computes mean of differences of the signal.
    features.append(median_abs_deviation(signal))  #Computes median absolute deviation of the signal.
    features.append(median_abs_diff(signal))   #Computes median absolute differences of the signal.
    features.append(median_diff(signal))   #Computes median of differences of the signal.
    features.append(median_frequency(signal, new_sRate))  #Computes median frequency of the signal.
    features.append(pk_pk_distance(signal))   #Computes the peak to peak distance.
    features.append(rms(signal))   #Computes root mean square of the signal.
    features.append(skewness(signal))  #Computes skewness of the signal.
    features.append(spectral_centroid(signal, new_sRate))     #Barycenter of the spectrum.
    features.append(spectral_decrease(signal, new_sRate))     #Represents the amount of decreasing of the spectra amplitude.
    features.append(spectral_distance(signal, new_sRate))     #Computes the signal spectral distance.
    features.append(spectral_kurtosis(signal, new_sRate))     #Measures the flatness of a distribution around its mean value.
    features.append(spectral_positive_turning(signal, new_sRate))     #Computes number of positive turning points of the fft magnitude signal.
    features.append(spectral_roll_off(signal, new_sRate))     #Computes the spectral roll-off of the signal.
    features.append(spectral_roll_on(signal, new_sRate))  #Computes the spectral roll-on of the signal.
    features.append(spectral_skewness(signal, new_sRate))     #Measures the asymmetry of a distribution around its mean value.
    features.append(spectral_slope(signal, new_sRate))    #Computes the spectral slope.
    features.append(spectral_spread(signal, new_sRate))   #Measures the spread of the spectrum around its mean value.
    features.append(spectral_variation(signal, new_sRate))    #Computes the amount of variation of the spectrum along time.
    features.append(sum_abs_diff(signal))  #Computes sum of absolute differences of the signal.
    features.append(total_energy(signal, new_sRate))  #Computes the total energy of the signal.
    
    return np.array(features)

In [31]:
def get_features(X):
    
    """
    Takes in a 3d array where the features are calculated along axis-2. 
    The features are calculated for every channel for every sample.
    The features of all channels are then flattened together for every sample.
    
    Input:
    X - Data array of shape (n_samples, n_channels, sample_size)
    
    Returns:
    X_features - Data array of shape (n_samples, n_features)
    """
    
    n_samples = X.shape[0]
    
    X = X.reshape(-1, sample_size)

    X_features = Parallel(n_jobs=-1)(delayed(calc_Features)(X[s]) for s in tqdm(range(X.shape[0])))

    X_features = np.array(X_features)

    n_features_per_vec = X_features.shape[1]

    X_features = X_features.reshape(n_samples, n_channels, n_features_per_vec)
    
    X_features = X_features.reshape(X_features.shape[0], -1)

    return X_features

### Feature extraction from EC condition data

In [ ]:
train_features_ec = get_features(X_train_ec_aug)

print("Training data feature shape for EC condition: ", train_features_ec.shape)

del(X_train_ec_aug)
gc.collect()

  0%|          | 0/1175040 [00:00<?, ?it/s]

In [ ]:
valid_features_ec = get_features(X_valid_ec_aug)

print("Validation data feature shape for EC condition: ", valid_features_ec.shape)

del(X_valid_ec_aug)
gc.collect()

In [ ]:
test_features_ec = get_features(X_test_ec_aug)

print("Test data feature shape for EC condition: ", test_features_ec.shape)

del(X_test_ec_aug)
gc.collect()

### Feature extraction from EO condition data

In [ ]:
train_features_eo = get_features(X_train_eo_aug)

print("Training data feature shape for EO condition: ", train_features_eo.shape)

del(X_train_eo_aug)
gc.collect()

In [ ]:
valid_features_eo = get_features(X_valid_eo_aug)

print("Validation data feature shape for EO condition: ", valid_features_eo.shape)

del(X_valid_eo_aug)
gc.collect()

In [ ]:
test_features_eo = get_features(X_test_eo_aug)

print("Test data feature shape for EO condition: ", test_features_eo.shape)

del(X_test_eo_aug)
gc.collect()

# Saving Extracted features for further training with AutoML

In [ ]:
Y_train_ec = np.expand_dims(Y_train_ec_aug, axis=1)
Y_valid_ec = np.expand_dims(Y_valid_ec_aug, axis=1)

Y_train_eo = np.expand_dims(Y_train_eo_aug, axis=1)
Y_valid_eo = np.expand_dims(Y_valid_eo_aug, axis=1)

print("Eyes closed Training target shape:", Y_train_ec.shape)
print("Eyes closed Validation target shape:", Y_valid_ec.shape)
print("\n")
print("Eyes open Training target shape:", Y_train_eo.shape)
print("Eyes open Validation target shape:", Y_valid_eo.shape)

### Saving for EC

In [ ]:
df_train_ec = pd.DataFrame(data=np.hstack((train_features_ec, Y_train_ec)), index=None, columns=None)
df_valid_ec = pd.DataFrame(data=np.hstack((valid_features_ec, Y_valid_ec)), index=None, columns=None)

df_train_ec.to_csv('exp1/ec/df_train_ec.csv', index=False)
df_valid_ec.to_csv('exp1/ec/df_valid_ec.csv', index=False)

print("Final training data shape for EC condition: ", df_train_ec.shape)
print("Final validation data shape for EC condition: ", df_train_ec.shape)

In [ ]:
print("Saving test data features EC condition: ", test_features_ec.shape)

with open('exp1/ec/test_ec.npy', 'wb') as f:
    np.save(f, test_features_ec)

### Saving for EO

In [ ]:
df_train_eo = pd.DataFrame(data=np.hstack((train_features_eo, Y_train_eo)), index=None, columns=None)
df_valid_eo = pd.DataFrame(data=np.hstack((valid_features_eo, Y_valid_eo)), index=None, columns=None)

df_train_eo.to_csv('exp1/eo/df_train_eo.csv', index=False)
df_valid_eo.to_csv('exp1/eo/df_valid_eo.csv', index=False)

print("Final training data shape for EO condition: ", df_train_eo.shape)
print("Final validation data shape for EO condition: ", df_train_eo.shape)

In [ ]:
print("Saving test data features EO condition: ", test_features_eo.shape)

with open('exp1/eo/test_eo.npy', 'wb') as f:
    np.save(f, test_features_eo)

### Saving Feature Extraction Info

In [ ]:
with open('exp1/feature_extraction_info.json', 'w') as f:
    json.dump(feature_extraction_info, f, sort_keys = True, indent = 4, ensure_ascii = False)